In [13]:
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import BatchNormalization
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
import re

In [14]:
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package omw-1.4 to /home/d81v711/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/d81v711/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/d81v711/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
# initialize the files
words = [] #words have tpkenized words from pattern
classes = []  # tags
documents = [] # tags + tokenized words
ignore_words = ["?", "!"]
data_file = open("/home/d81v711/intents.json").read()
intents = json.loads(data_file)

In [16]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [17]:
##################### new addition##########
#pattern = pattern = r'\b\w\b'
#words = [re.sub(pattern, '', w) for w in words]
##############

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))


196 documents
75 classes ['AI', 'abbr', 'action', 'actions', 'address', 'age', 'artificial', 'available_tables', 'bad', 'bend', 'body', 'book_table', 'bot1', 'breathe', 'business', 'chatbot', 'chatterbox', 'city', 'clone', 'comp', 'computer', 'contact', 'control', 'cramped', 'date', 'death', 'do', 'events', 'fav', 'feeling', 'fight', 'good', 'goodbye', 'greeting', 'greetings', 'hardware', 'hello', 'hobby', 'hours', 'idea', 'imortal', 'job', 'lang', 'laugh', 'lie', 'machine', 'malfunction', 'men', 'menu', 'motormouth', 'move', 'name', 'name1', 'need', 'noanswer', 'os', 'program', 'programming', 'ratchet', 'robotics', 'robots', 'robotss', 'sapient', 'sense', 'sentiment', 'shoe', 'sound', 'still there', 'stupid', 'thanks', 'usage', 'wait', 'who', 'women', 'wt']
207 unique lemmatized words ["'m", "'s", ',', '2', 'a', 'address', 'afternoon', 'age', 'ai', 'all', 'allowed', 'am', 'an', 'anyone', 'are', 'artificial', 'ask', 'available', 'awesome', 'bad', 'be', 'being', 'bend', 'body', 'book', 

In [18]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        if w in pattern_words:
            print("word:" ,w)
            bag.append(1) 
        else:
            bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    tag_index = output_row.index(1)
    tag_value = classes[tag_index]
    print("Tag for this word:",tag_value)
    training.append([bag, output_row])

print("Training dataset size: ", len(training))
    
for i in range(2):
    print("Row", i+1)
    print("Bag of Words:", training[i][0])
    print("Output Row:", training[i][1])
    print()    
    
# shuffle our features and turn into np.array
random.shuffle(training)

# Separate bag-of-words representations and output labels
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

#print(len(train_y))
# Convert to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)
print("Training data created")

word: hello
Tag for this word: hello
word: hi
word: there
Tag for this word: hello
word: good
word: morning
Tag for this word: hello
word: 's
word: up
word: what
Tag for this word: hello
word: is
word: job
word: what
word: your
Tag for this word: job
word: is
word: what
word: work
word: your
Tag for this word: job
word: age
word: is
word: what
word: your
Tag for this word: age
word: are
word: how
word: old
word: you
Tag for this word: age
word: born
word: were
word: when
word: you
Tag for this word: age
word: are
word: how
word: today
word: you
Tag for this word: feeling
word: are
word: how
word: you
Tag for this word: feeling
word: am
word: good
word: i
word: too
Tag for this word: good
word: feel
word: fine
word: i
Tag for this word: good
word: good
Tag for this word: good
word: fine
Tag for this word: good
word: am
word: good
word: i
Tag for this word: good
word: am
word: great
word: i
Tag for this word: good
word: great
Tag for this word: good
word: am
word: bad
word: feeling
word:

In [19]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               26624     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 75)                4875      
                                                                 
Total params: 39755 (155.29 KB)
Trainable params: 39755 (155.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [21]:
# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(train_x, train_y)
print("Training Accuracy:", train_accuracy)

model.save("chatbot_model.h5", hist)
print("model created")

Epoch 1/200
40/40 [==============================] - 0s 952us/step - loss: 4.3272 - accuracy: 0.0153
Epoch 2/200
40/40 [==============================] - 0s 890us/step - loss: 4.2038 - accuracy: 0.0510
Epoch 3/200
40/40 [==============================] - 0s 910us/step - loss: 4.1222 - accuracy: 0.0510
Epoch 4/200
40/40 [==============================] - 0s 916us/step - loss: 4.0100 - accuracy: 0.0612
Epoch 5/200
40/40 [==============================] - 0s 914us/step - loss: 3.8928 - accuracy: 0.0867
Epoch 6/200
40/40 [==============================] - 0s 919us/step - loss: 3.7996 - accuracy: 0.0816
Epoch 7/200
40/40 [==============================] - 0s 920us/step - loss: 3.7379 - accuracy: 0.0969
Epoch 8/200
40/40 [==============================] - 0s 923us/step - loss: 3.5526 - accuracy: 0.1633
Epoch 9/200
40/40 [==============================] - 0s 900us/step - loss: 3.4193 - accuracy: 0.1429
Epoch 10/200
40/40 [==============================] - 0s 909us/step - loss: 3.2862 - accura

/home/d81v711/.conda/envs/myenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
#load test file
wordsT= [] #words have tpkenized words from pattern
classesT= []  # tags
documentsT= [] # tags + tokenized words
ignore_words = ["?", "!"]
data_fileTest = open("/home/d81v711/intents_test.json").read()
intents_test = json.loads(data_fileTest)
testing = []

#make bag of words for the patterns in the test file.
for inte in intents_test["intents"]:
    for patt in inte["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(patt)
        wordsT.extend(w)
        # adding documents
        documentsT.append((w, inte["tag"]))

        # adding classes to our class list
        if inte["tag"] not in classesT:
            classesT.append(inte["tag"])

output_empty = [0] * len(classes)
for doc in documentsT:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_wordsT = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_wordsT = [lemmatizer.lemmatize(word.lower()) for word in pattern_wordsT]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        if w in pattern_wordsT:
            #print("word:" ,w)
            bag.append(1) 
        else:
            bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1


    #print("Tag for this word:",tag_value)
    testing.append([bag, output_row])


print("Testing dataset size: ", len(testing))
    
for i in range(2):
    print("Row", i+1)
    print("Bag of Words:", testing[i][0])
    print("Output Row:", testing[i][1])
    print()    
    
    
# Separate bag-of-words representations and output labels
test_x = [item[0] for item in testing]
test_y = [item[1] for item in testing]

#print(len(train_y))
# Convert to NumPy arrays
test_x = np.array(test_x)
test_y = np.array(test_y)

print("Testing data created")                


Testing dataset size:  35
Row 1
Bag of Words: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output Row: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Row 2
Bag of Words: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [23]:

y_pred = model.predict(test_x)

ERROR_THRESHOLD = 0.25
return_list = []
print("Length of testing data",len(y_pred))
count = -1
for y_p in y_pred:
    count = count +1
    results = [[i, r] for i, r in enumerate(y_p) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)

    if results:
        r = results[0] 
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

        
#see if the intent matches with the original intent 
count = 0
accuracy = 0
actual_label = []
predicted_label = []
#print(len(return_list))
for item in return_list:
    # print(test_y[count])
    index = np.argwhere(test_y[count] == 1).flatten()
    actual_label.append( classes[index[0]])
    # Get the corresponding tag from the classes list
    predicted_label.append(item["intent"])
    if classes[index[0]] == item["intent"]:
        #print("these two are equal")
        accuracy = accuracy + 1
    count= count+1
    
print("Accuracy:", (accuracy / len(y_pred)) *100)

conf_matrix = confusion_matrix(actual_label, predicted_label)


# Confusion matrix
print("Confusion matrix:")
print(conf_matrix)


2/2 [==============================] - 0s 2ms/step
Length of testing data 35
Accuracy: 62.857142857142854
Confusion matrix:
[[1 0 0 0 2 0 0 0 0 0 0 0 1 0 0]
 [0 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 2 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 2]]


In [24]:
model = load_model("chatbot_model.h5")
data_file = open("intents.json").read()
intents = json.loads(data_file)
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

In [25]:
# chat functionalities
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    #print(sentence)
    res = model.predict(np.array([p]))[0]
    #print(res)
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    #print(return_list)
    return return_list


def getResponse(ints, intents_json):
    if not ints:
        return "I am sorry, I do not have answer to this question"
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

def chatbot_response(question):
    ints = predict_class(question, model)
    res = getResponse(ints, intents)
    return res

answer = chatbot_response("Computer are good")

1/1 [==============================] - 0s 38ms/step


Computer are good
1/1 [==============================] - 0s 15ms/step


In [ ]:
# Start chatbot
while True:
    question = input('You: ')
    answer = chatbot_response(question)
    print('Chatbot:', answer)

You:  Hi how are you doing today


1/1 [==============================] - 0s 13ms/step
Chatbot: Actually, I'm okay and you?


You:  are you a robot


1/1 [==============================] - 0s 13ms/step
Chatbot: Sure, I am a women


You:  how old are you


1/1 [==============================] - 0s 13ms/step
Chatbot: I was born in 2021
